In [ ]:
# --- Step 1: 環境設定 ---
import os
import time

print(">>> Step 1: 環境設定を開始します...")

# タイムゾーンを日本時間(JST)に設定
os.environ['TZ'] = 'Asia/Tokyo'
time.tzset()
print(f"  - タイムゾーンをJSTに設定しました。 現在時刻: {time.strftime('%Y-%m-%d %H:%M:%S')}")

# システムパッケージのインストール
print("  - 必要なシステムパッケージをインストール中...")
!apt-get update -qq 2>/dev/null
!apt-get install -y chromium-chromedriver fonts-ipafont-gothic -qq > /dev/null 2>&1

# Pythonライブラリのインストール
print("  - 必要なPythonライブラリをインストール中...")
!pip install selenium pandas pillow tqdm -q --no-warn-conflicts 2>/dev/null

print("\n✅ 環境設定が完了しました。次のステップに進んでください。")

In [ ]:
# --- Step 2: 自治体リストのアップロード ---
from google.colab import files

print(">>> Step 2: 自治体リストのCSVファイルをアップロードしてください。")
print("（ファイル形式: 自治体コード,自治体名,URL）")

uploaded = files.upload()

if not uploaded:
    print("\n⚠️ ファイルがアップロードされませんでした。処理を中断します。")
else:
    uploaded_file_name = list(uploaded.keys())[0]
    print(f"\n✅ ファイル '{uploaded_file_name}' が正常にアップロードされました。")
    print("次のステップに進んでください。")

In [ ]:
# --- Step 3: 実行準備 ---
# このセルは、スクリーンショット取得の全工程を管理する関数を定義します。

import os, pandas as pd, shutil, random, time
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, WebDriverException
from PIL import Image
from tqdm import tqdm

print(">>> Step 3: メイン処理のプログラムを準備します...")

# --- 基本設定 ---
OUTPUT_BASE_DIR = 'output'
SCREENSHOTS_DIR = os.path.join(OUTPUT_BASE_DIR, 'screenshots') # 保存先は単一のフォルダ
RESULT_CSV_PATH = os.path.join(OUTPUT_BASE_DIR, 'results.csv')
ARCHIVE_NAME_BASE = 'municipality_screenshots'
ARCHIVE_PATH = f'{ARCHIVE_NAME_BASE}.zip'
RESIZED_WIDTH, RESIZED_HEIGHT = 1920, 1080
BROWSER_WINDOW_WIDTH, BROWSER_WINDOW_HEIGHT = 1920, 1080
PAGE_LOAD_TIMEOUT = 30

def read_flexible_csv(file_path):
    # (変更なし)
    try:
        with open(file_path, 'r', encoding='utf-8') as f: first_line = f.readline()
        encoding = 'utf-8'
    except UnicodeDecodeError:
        try:
            with open(file_path, 'r', encoding='shift_jis') as f: first_line = f.readline()
            encoding = 'shift_jis'
        except Exception as e: return None, f"ファイルをUTF-8またはShift_JISで読み込めませんでした。{e}"
    has_header = not first_line.split(',')[0].strip().strip('"').isdigit()
    try:
        if has_header:
            df = pd.read_csv(file_path, encoding=encoding, header=0, usecols=[0, 1, 2], names=['code', 'name', 'url'], dtype={'code': str})
        else:
            df = pd.read_csv(file_path, encoding=encoding, header=None, usecols=[0, 1, 2], names=['code', 'name', 'url'], dtype={'code': str})
        return df, f"ファイル情報を検出: [文字コード: {encoding}, ヘッダー: {'あり' if has_header else 'なし'}]"
    except Exception as e: return None, f"CSVの解析中にエラーが発生しました: {e}"

def setup_driver():
    # (変更なし)
    options = webdriver.ChromeOptions()
    options.add_argument('--headless'); options.add_argument('--no-sandbox'); options.add_argument('--disable-dev-shm-usage')
    options.add_argument(f'--window-size={BROWSER_WINDOW_WIDTH},{BROWSER_WINDOW_HEIGHT}')
    options.add_argument('--ignore-certificate-errors'); options.add_argument('--allow-running-insecure-content')
    driver = webdriver.Chrome(options=options); driver.set_page_load_timeout(PAGE_LOAD_TIMEOUT)
    return driver

def main_process(input_csv_path):
    """スクリーンショット取得の全工程を管理するメイン関数"""
    df, message = read_flexible_csv(input_csv_path)
    print(message);
    if df is None: return print(f"❌ 処理を中断します。({message})"), False

    print(f"合計 {len(df)} 件のデータを読み込みました。")

    driver = setup_driver()
    os.makedirs(SCREENSHOTS_DIR, exist_ok=True) # 保存用フォルダを一つだけ作成
    results = []

    print("スクリーンショットの取得を開始します。件数によっては長時間かかります...")

    for _, row in tqdm(df.iterrows(), total=len(df), desc="スクリーンショット取得中"):
        code, name, url = row['code'], row['name'], row['url']
        base_info = {'code': code, 'name': name, 'url': url} # 結果記録用の基本情報

        # ファイルパスは screenshots フォルダ直下に設定
        temp_path = os.path.join(SCREENSHOTS_DIR, f"{code}_temp.png")
        final_path = os.path.join(SCREENSHOTS_DIR, f"{code}_{name}.png")

        try:
            # --- シンプルなアクセス処理 ---
            # 提供されたURLに一度だけアクセスを試みる
            tqdm.write(f"  -> {name} ({url}) にアクセス中...")
            driver.get(url)
            time.sleep(random.uniform(2, 5)) # ページ読み込み待機

            # 表示された画面をそのまま撮影（サイト本体でもエラー画面でも）
            driver.save_screenshot(temp_path)
            with Image.open(temp_path) as img:
                if img.mode in ('RGBA', 'P'): img = img.convert('RGB')
                img.resize((RESIZED_WIDTH, RESIZED_HEIGHT), Image.Resampling.LANCZOS).save(final_path)
            os.remove(temp_path)

            # アクセスと撮影に成功したので「成功」として記録
            results.append({'status': '成功', 'error': '', **base_info})

        except (TimeoutException, WebDriverException) as e:
            # タイムアウトなどでアクセス自体に失敗した場合
            error_message = str(e).split('\n')[0]
            tqdm.write(f"  -> 失敗: {name} ({error_message})")
            results.append({'status': '失敗', 'error': error_message, **base_info})

    driver.quit(); print("\n全てのスクリーンショット処理が完了しました。")

    pd.DataFrame(results).to_csv(RESULT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"処理結果レポートを '{RESULT_CSV_PATH}' に保存しました。")
    print("成果物をZIPファイルに圧縮しています..."); shutil.make_archive(ARCHIVE_NAME_BASE, 'zip', OUTPUT_BASE_DIR)
    return True

print("✅ メイン処理の準備が完了しました。次のステップに進んでください。")

In [ ]:
# --- Step 4: 処理実行とダウンロード ---
from google.colab import files

if 'uploaded_file_name' not in locals() or 'main_process' not in globals():
    print("❌ エラー: Step 2とStep 3を先に実行してください。")
else:
    success = main_process(uploaded_file_name)

    if success:
        print(f"\n✅ 全ての処理が完了しました！ 成果物は '{ARCHIVE_PATH}' として保存されています。")
        print("\nこれからファイルのダウンロードを開始します...")

        try:
            files.download(ARCHIVE_PATH)
            print("✅ ダウンロードが開始されました。ブラウザの指示に従ってください。")
        except Exception as e:
            print(f"⚠️ 自動ダウンロードに失敗しました: {e}")

        print("\n---")
        print("【バックアップ案内】")
        print("もし自動でダウンロードが始まらない場合は、以下の手順で手動ダウンロードが可能です：")
        print(f"1. 画面左のファイルブラウザ（フォルダのアイコン）で更新ボタンを押します。")
        print(f"2. 表示された '{ARCHIVE_PATH}' を右クリックし、「ダウンロード」を選択してください。")